### test_brisque_scores

Brisque (blind/referenceless image spatial quality evaluator) scores are a way of evaluating image quality.

This test uses the package at https://pypi.org/project/image-quality/   
but I did `conda install image-quality`

Here is a nice notebook that does all of the calculations from scratch: https://github.com/ocampor/notebooks/blob/master/notebooks/image/quality/brisque.ipynb

Mittal, A., Moorthy, A. K., & Bovik, A. C. (2012). No-Reference Image Quality Assessment in the Spatial Domain. IEEE Transactions on Image Processing, 21(12), 4695–4708. https://doi.org/10.1109/TIP.2012.2214050


In [1]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io, img_as_float
from skimage.transform import rescale

import glob
import imquality.brisque as brisque

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
%%time
impath = r'..\data\1581508801.c1.timex.jpg'
img = img_as_float(io.imread(impath, as_gray=True))
bs = brisque.score(img)
print('Brisque score:',bs)

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)
C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


Brisque score: 45.26572446406166
Wall time: 40 s


In [3]:
%%time
impath = r'..\data\1581508801.c1.timex.jpg'
img = rescale(img_as_float(io.imread(impath, as_gray=True)),.5,anti_aliasing=True)
bs = brisque.score(img)
print('Brisque score, scale factor 0.5:',bs)

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


Brisque score, scale factor 0.5: 27.297608834122997
Wall time: 10.7 s


In [4]:
%%time
impath = r'..\data\1581508801.c1.timex.jpg'
img = rescale(img_as_float(io.imread(impath, as_gray=True)),.25,anti_aliasing=True)
bs = brisque.score(img)
print('Brisque score, scale factor 0.25:',bs)

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


Brisque score, scale factor 0.25: 20.209473350875868
Wall time: 3.17 s


In [5]:
# this takes hours - comment it out for now
if(False):
    ofile = open('brisque_scores.csv','w')
    for i, f in enumerate( glob.glob(r'C:\crs\proj\2019_CACO_CoastCam\random_obliques\*.jpg') ):
        img = img_as_float(io.imread(f, as_gray=True))
        bs = brisque.score(img)
        ts = '{}, {:.3f}\n'.format(f,bs)
        print(ts)
        ofile.write(ts)
    #     plt.imshow(img,cmap='gray')
    #     plt.title('BRISQUE score {:.2f}'.format(bs))
    ofile.close()


In [6]:
# This is much faster...but still commented out because we have already done it
if(False):
    ofile2 = open('brisque_scores_small.csv','w')
    for i, f in enumerate( glob.glob(r'C:\crs\proj\2019_CACO_CoastCam\random_obliques\*.jpg') ):
        img = img_as_float(io.imread(f, as_gray=True))
        imgs = rescale(img,0.25,anti_aliasing=True)
        bs = brisque.score(imgs)
        ts = '{}, {:.3f}\n'.format(f,bs)
        print(ts)
        ofile2.write(ts)
    ofile2.close()

In [7]:
# this should be imported from coastcam_funcs, but for convenience, I put it here
from skimage.color import rgb2gray
def estimate_sharpness(img):
    """
    Estimate image sharpness and contrast
    Input:
        img - np.ndarray representing image
              img read as skimage.io.imread( 'imagefilename.jpg' )
    Returns:
        s,c - sharpness and contrast estimates

    https://stackoverflow.com/questions/6646371/detect-which-image-is-sharper
    """
    array = np.asarray(rgb2gray(img), dtype=np.int32)
    contrast = array.std()
    gy, gx = np.gradient(array)
    gnorm = np.sqrt(gx**2 + gy**2)
    sharpness = np.average(gnorm)
    return sharpness, contrast

In [8]:
# calculate sharpness at two resolutions
ofile3 = open('sharpness_scores_small.csv','w')
for i, f in enumerate( glob.glob(r'C:\crs\proj\2019_CACO_CoastCam\random_obliques\*.jpg') ):
    img = img_as_float(io.imread(f, as_gray=True))
    s, c = estimate_sharpness(img)
    imgs = rescale(img,0.25,anti_aliasing=True)
    ss, cs = estimate_sharpness(imgs)
    ts = '{}, {:.6f}, {:.6f}, {:.6f}, {:.6f}\n'.format(f,s,ss,c,cs)
    print(ts)
    ofile3.write(ts)
ofile3.close()

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576261801.c1.timex.jpg, 0.000212, 0.000000, 0.011429, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576261801.c2.timex.jpg, 0.000203, 0.000000, 0.011066, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576504801.c1.timex.jpg, 0.000116, 0.000000, 0.008125, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576504801.c2.timex.jpg, 0.000876, 0.000035, 0.043771, 0.005053

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576508401.c1.timex.jpg, 0.000215, 0.000000, 0.011455, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576508401.c2.timex.jpg, 0.000184, 0.000000, 0.010811, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1577188801.c1.timex.jpg, 0.001481, 0.002958, 0.137372, 0.105362

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1577188801.c2.timex.jpg, 0.001634, 0.003786, 0.200063, 0.179117

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1577219401.c1.timex.jpg, 0.000162, 0.000000, 0.010066, 0.

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1586457001.c1.timex.jpg, 0.000249, 0.000000, 0.012520, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1586457001.c2.timex.jpg, 0.000163, 0.000000, 0.009764, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1587040201.c1.timex.jpg, 0.000048, 0.000000, 0.005092, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1587040201.c2.timex.jpg, 0.000136, 0.000000, 0.008694, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1587837601.c1.timex.jpg, 0.000145, 0.000000, 0.009325, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1587837601.c2.timex.jpg, 0.000164, 0.000000, 0.010006, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1588336201.c1.timex.jpg, 0.000184, 0.000000, 0.010625, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1588336201.c2.timex.jpg, 0.000253, 0.000000, 0.012448, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1588446001.c1.timex.jpg, 0.000144, 0.000000, 0.009442, 0.

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594328401.c1.timex.jpg, 0.000415, 0.000485, 0.073465, 0.063981

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594328401.c2.timex.jpg, 0.000097, 0.000000, 0.007645, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594546201.c1.timex.jpg, 0.000156, 0.000000, 0.009641, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594546201.c2.timex.jpg, 0.001342, 0.001816, 0.107162, 0.068517

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594724401.c1.timex.jpg, 0.000045, 0.000000, 0.004810, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594724401.c2.timex.jpg, 0.001286, 0.003189, 0.204559, 0.187948

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594927801.c1.timex.jpg, 0.000185, 0.000000, 0.010737, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1594927801.c2.timex.jpg, 0.000196, 0.000000, 0.011120, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1595192401.c1.timex.jpg, 0.000189, 0.000000, 0.010709, 0.

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602174601.c1.timex.jpg, 0.000101, 0.000000, 0.007951, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602174601.c2.timex.jpg, 0.000118, 0.000000, 0.008497, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602417601.c1.timex.jpg, 0.001444, 0.004176, 0.261083, 0.245649

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602417601.c2.timex.jpg, 0.001574, 0.003708, 0.179294, 0.155973

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602529201.c1.timex.jpg, 0.000157, 0.000000, 0.009865, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602529201.c2.timex.jpg, 0.000203, 0.000000, 0.011003, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602603001.c1.timex.jpg, 0.000165, 0.000000, 0.010184, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1602603001.c2.timex.jpg, 0.000194, 0.000000, 0.010662, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1603287001.c1.timex.jpg, 0.000148, 0.000000, 0.009250, 0.

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1610890201.c1.timex.jpg, 0.000140, 0.000000, 0.009142, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1610890201.c2.timex.jpg, 0.001029, 0.000793, 0.078983, 0.052680

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611091801.c1.timex.jpg, 0.000225, 0.000000, 0.011832, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611091801.c2.timex.jpg, 0.000246, 0.000000, 0.012263, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611241201.c1.timex.jpg, 0.000184, 0.000000, 0.010838, 0.000000

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611241201.c2.timex.jpg, 0.000222, 0.000000, 0.011966, 0.000000

